# Exploration of well based metrics

We first load some packages

In [98]:
import polars as pl
import plotly.express as px
import sklearn
import umap

/home/adelabriere/miniconda3/envs/imaging/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html




We first specify the aggregated path by well

In [1]:
PATH_FILE = "/mnt/c/Users/alexi/Documents/data/images/cellpainting/cpg0016-jump/data/results_target/well_descriptors.csv"

We load the datasets

In [ ]:
df = pl.read_csv(PATH_FILE)

/tmp/ipykernel_30340/4067332876.py:6: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  pl.select(pl.col(pl.NUMERIC_DTYPES))


In [35]:

# Drop all string columns
dfr = df.select(pl.col(pl.NUMERIC_DTYPES))

/tmp/ipykernel_30340/45389431.py:2: DeprecationWarning: `NUMERIC_DTYPES` was deprecated in version 1.0.0. Define your own data type groups or use the `polars.selectors` module for selecting columns of a certain data type.
  dfr = df.select(pl.col(pl.NUMERIC_DTYPES))


We first check the coefficient of cariation of the different values

In [38]:
df_stats = dfr.describe()

In [64]:
dmean = df_stats.filter(df_stats["statistic"]=="mean").drop(pl.col("statistic"))
dstd = df_stats.filter(df_stats["statistic"]=="std").drop(pl.col("statistic"))
dcv = dstd/dmean


In [67]:
dcv = dcv.transpose(include_header=True).rename({"column": "variable", "column_0": "cv"})

In [77]:
px.histogram(dcv,x="cv")

We opnly keep the the top 1000 variables for visualization purpose

In [79]:
sel_dcv = dcv.filter(pl.col("cv").is_in(pl.col("cv").top_k(1000)))
sel_vars = sel_dcv["variable"].to_list()

/tmp/ipykernel_30340/2242901189.py:1: DeprecationWarning:

`is_in` with a collection of the same datatype is ambiguous and deprecated.
Please use `implode` to return to previous behavior.

See https://github.com/pola-rs/polars/issues/22149 for more information.



We subset the data

In [96]:
sdf = df[sel_vars]
sdf_metas = df[["WellPlate"]]

We can now check the variable using a PCA

In [ ]:
# We compute the PCA of the variables with 50 components
N_COMPONENTS = 50
pca = sklearn.decomposition.PCA(n_components=N_COMPONENTS)
sdf_pca = pca.fit_transform(sdf)

We cna compute the umap representation

In [100]:
um = umap.UMAP(n_components=2,n_neighbors=30)
sdf_umap = um.fit_transform(sdf_pca)

/home/adelabriere/miniconda3/envs/imaging/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning:

'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.



We add the representations of the metadata table

In [103]:
for ipc in range(N_COMPONENTS):
    sdf_metas = sdf_metas.with_columns(
        pl.lit(sdf_pca[:,ipc]).alias("PC"+str(ipc+1))
    )

for ium in range(2):
    sdf_metas =  sdf_metas.with_columns(
        pl.lit(sdf_umap[:,ium]).alias("UMAP"+str(ium+1))
    )

Reconstructing the well and plate information

In [113]:
sdf_metas = sdf_metas.with_columns(
    pl.col("WellPlate").str.split("_").list.get(-1).alias("Well"),
    pl.col("WellPlate").str.split("_").list.get(0).alias("Plate"),
    pl.col("Well").str.extract(r"([A-Z]+)([0-9]+)",group_index=1).alias("WellRow"),
    pl.col("Well").str.extract(r"([A-Z]+)([0-9]+)",group_index=2).alias("WellCol")
)

Plotting representation

In [120]:
px.scatter(sdf_metas,x="UMAP1",y="UMAP2",color="Plate")